This notebook will describe the methods used to generate a control dataset for benchmarking the pipeline against a set of structures said to be in a **'parent-child'** relationship. Before proceeding to the methods, the definition of what structural relationships fall into this category will be presented.

**Parent-child Relationship**

Any pair of structures where structure A has more defined stereochemistry than structure A', indicating that A encapsulates A'—that is, A' represents a less stereochemically defined version of A. This includes cases where A is fully defined while A' lacks stereochemical definition entirely, as well as cases where both are partially defined but A retains greater stereochemical detail.

**Hypothesis**: pipeline will assign "2D vs 3D" classification for majority of the parent-child relationships extracted from MetaNetX


Extra note: could just use the set of defined parent-children by metanetx as benchmarking test, at least for now as it will be much quicker. Only consideration here is that we cannot use the results of comparing the relationships between the two really then? or can we? Do ths for now, discuss with peers and PI and revisit based on discussions.

In [ ]:
from rdkit import Chem
import pandas as pd
import os
import subprocess
import sqlite3
import json
from stereomapper.domain.chemistry import ChemistryUtils
from pathlib import Path

In [ ]:
# copy the unique structures to a new directory for processing


In [ ]:
# run stereomapper as a subprocess
parent_child = Path.home('parent_child_benchmark_data')
results = Path.home('parent_child_benchmark_result.sqlite')
cache_path = Path.home('parent_child_benchmark_cache.sqlite') 

cmd =[
    "stereomapper",
    "run",
    "-d", results.as_posix(),
    "-o", results.as_posix(),
    "-p", cache_path.as_posix(),
    "--fresh-cache"
]

subprocess.run(cmd) 

INFO    Logging initialised. File: /media/JACK/stereomapper_logs/stereomapper_20251104_110541.log
Pipeline: 100%|██████████████████████████████████████████████████████████| , Complete! [01:00<00:00]


✅ Pipeline completed in 60.2s
📦 Inputs attempted: 7,693 (skipped 0)
📊 Successes: 7,693 | Failures: 0
🔗 Inchikey groups — processed 2,541, skipped 0, failed 0
🧮 Relationship rows: 15,257
🧾 Unique inchikeys observed: 2,541
💾 Cache hit rate: 0.0%


Pipeline: 100%|██████████████████████████████████████████████████████████| , Complete! [01:00<00:00]


CompletedProcess(args=['stereomapper', 'run', '-d', '/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/data/stereomapper/control_sets/stereoisomer_control/parent_child_benchmark_data', '-o', '/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/results/stereomapper/project_results/benchmarking/041125_parent_child_benchmark.sqlite', '-p', '/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/results/stereomapper/project_results/benchmarking/cache/041125_parent_child_benchmark.sqlite', '--fresh-cache'], returncode=0)

In [14]:
# now open up the results db 
conn = sqlite3.connect(results.as_posix())
df_results = pd.read_sql_query("SELECT * FROM relationships", conn)
df_results

,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
0,1,2,"[""chebi:CHEBI:184417""]","[""chebi:CHEBI:81326""]",1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
1,1,3,"[""chebi:CHEBI:184417""]","[""chebi:CHEBI:81325""]",1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
2,2,3,"[""chebi:CHEBI:81326""]","[""chebi:CHEBI:81325""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3,4,5,"[""chebi:CHEBI:20604""]","[""chebi:CHEBI:40617""]",1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0
4,4,6,"[""chebi:CHEBI:20604""]","[""chebi:CHEBI:52505""]",1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
...,...,...,...,...,...,...,...,...,...,...,...
15252,7680,7683,"[""chebi:CHEBI:138363""]","[""chebi:CHEBI:138362""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
15253,7681,7682,"[""chebi:CHEBI:138597""]","[""chebi:CHEBI:138596""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
15254,7681,7683,"[""chebi:CHEBI:138597""]","[""chebi:CHEBI:138362""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
15255,7682,7683,"[""chebi:CHEBI:138596""]","[""chebi:CHEBI:138362""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0


We see a lot of rows here, when we only expect to have 5,068 rows, according to the number of pairs extracted in above cells. To allow for correct comparison, need to cross-reference back on to the original rows, to get an accurate reperesentation of the performance of the pipeline. 

In [15]:
# load json lists from the results
df_results['cluster_a_members'] = df_results['cluster_a_members'].apply(json.loads)
df_results['cluster_b_members'] = df_results['cluster_b_members'].apply(json.loads)
df_results

,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
0,1,2,[chebi:CHEBI:184417],[chebi:CHEBI:81326],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
1,1,3,[chebi:CHEBI:184417],[chebi:CHEBI:81325],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
2,2,3,[chebi:CHEBI:81326],[chebi:CHEBI:81325],1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3,4,5,[chebi:CHEBI:20604],[chebi:CHEBI:40617],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0
4,4,6,[chebi:CHEBI:20604],[chebi:CHEBI:52505],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
...,...,...,...,...,...,...,...,...,...,...,...
15252,7680,7683,[chebi:CHEBI:138363],[chebi:CHEBI:138362],1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
15253,7681,7682,[chebi:CHEBI:138597],[chebi:CHEBI:138596],1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
15254,7681,7683,[chebi:CHEBI:138597],[chebi:CHEBI:138362],1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
15255,7682,7683,[chebi:CHEBI:138596],[chebi:CHEBI:138362],1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0


In [16]:
def strip_prefix(x):
    if isinstance(x, list):
        return [s.replace('chebi:CHEBI:', 'chebi:') for s in x]
    if isinstance(x, str):
        return x.replace('chebi:CHEBI:', 'chebi:')
    return x

df_results['cluster_a_members'] = df_results['cluster_a_members'].apply(strip_prefix)
df_results['cluster_b_members'] = df_results['cluster_b_members'].apply(strip_prefix)
df_results

,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
0,1,2,[chebi:184417],[chebi:81326],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
1,1,3,[chebi:184417],[chebi:81325],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
2,2,3,[chebi:81326],[chebi:81325],1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3,4,5,[chebi:20604],[chebi:40617],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0
4,4,6,[chebi:20604],[chebi:52505],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
...,...,...,...,...,...,...,...,...,...,...,...
15252,7680,7683,[chebi:138363],[chebi:138362],1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
15253,7681,7682,[chebi:138597],[chebi:138596],1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
15254,7681,7683,[chebi:138597],[chebi:138362],1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
15255,7682,7683,[chebi:138596],[chebi:138362],1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0


In [17]:
# now the data fromats match, create unique keys on both dataframes to allow for cross-referencing
df_parent_child['pair_key'] = df_parent_child.apply(lambda row: frozenset([row['parent_label'], row['child_label']]), axis=1)
df_results['pair_key'] = df_results.apply(lambda row: frozenset(row['cluster_a_members'] + row['cluster_b_members']), axis=1)

# now we can cross-reference the results back to the original set of parent-child pairs
merged_df = pd.merge(df_parent_child, df_results, on='pair_key', how='inner', suffixes=('_parent_child', '_results'))
print(f"Number of matched parent-child pairs in results: {merged_df.shape[0]}")

Number of matched parent-child pairs in results: 5053


In [18]:
frac_found = merged_df.shape[0] / df_parent_child.shape[0]
print(f"Fraction of parent-child pairs found in results: {frac_found:.2%}")

Fraction of parent-child pairs found in results: 99.39%


some missing, possibly due to errors in pipeline

In [19]:
merged_df

,mnxparent_label,parent_label,mnxchild_label,child_label,pair_key,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
0,MNXM100051,chebi:187719,MNXM100344,chebi:188242,"(chebi:187719, chebi:188242)",6897,6898,[chebi:187719],[chebi:188242],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
1,MNXM10010,chebi:28254,MNXM1380605,chebi:198196,"(chebi:198196, chebi:28254)",6790,6791,[chebi:28254],[chebi:198196],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
2,MNXM10026,chebi:86041,MNXM1104681,chebi:62616,"(chebi:62616, chebi:86041)",2114,2116,[chebi:86041],[chebi:62616],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
3,MNXM10026,chebi:86041,MNXM9497,chebi:74272,"(chebi:74272, chebi:86041)",2114,2115,[chebi:86041],[chebi:74272],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
4,MNXM10039,chebi:50168,MNXM36497,chebi:50170,"(chebi:50170, chebi:50168)",253,257,[chebi:50168],[chebi:50170],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5048,MNXM9879,chebi:173438,MNXM1409533,chebi:228257,"(chebi:173438, chebi:228257)",5084,5085,[chebi:173438],[chebi:228257],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0
5049,MNXM9987,chebi:27389,MNXM162802,chebi:57731,"(chebi:57731, chebi:27389)",4821,4823,[chebi:27389],[chebi:57731],1,1,Parent-child,84.0,"{""confidence_bin"":""medium""}",None,v1.0
5050,MNXM9987,chebi:27389,MNXM732376,chebi:58655,"(chebi:27389, chebi:58655)",4821,4822,[chebi:27389],[chebi:58655],1,1,Parent-child,84.0,"{""confidence_bin"":""medium""}",None,v1.0
5051,MNXM99960,chebi:186405,MNXM101734,chebi:195754,"(chebi:195754, chebi:186405)",1057,1058,[chebi:186405],[chebi:195754],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0


In [20]:
# check diff classifications
merged_df['classification'].value_counts()

classification
Parent-child     4338
Unclassified      711
Diastereomers       3
Unresolved          1
Name: count, dtype: int64

vast majority sorted into 2D vs 3D as expected, followed by No classification.

Looks good! Should investigate the no classifications briefly, and double check the 4 structure pairs that make up diasteroeomers and identical structures!!

In [21]:
df_miss_dia = merged_df[merged_df['classification'] == 'Diastereomers']
df_miss_dia

,mnxparent_label,parent_label,mnxchild_label,child_label,pair_key,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
1750,MNXM1369131,chebi:156163,MNXM33456,chebi:185064,"(chebi:185064, chebi:156163)",921,922,[chebi:185064],[chebi:156163],1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
2145,MNXM1374129,chebi:182512,MNXM734257,chebi:31053,"(chebi:31053, chebi:182512)",7228,7229,[chebi:182512],[chebi:31053],1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
2240,MNXM1376176,chebi:188778,MNXM1364249,chebi:32903,"(chebi:188778, chebi:32903)",5461,5462,[chebi:32903],[chebi:188778],1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0


In the case of (chebi:182512, chebi:31053) and (chebi:188778, chebi:32903) these are in fact diastereomers. In the case of (chebi:156163, chebi:185064) this is a case where it appears the pipeline may in fact be incorrect!

In [22]:
df_ident = merged_df[merged_df['classification'] == 'Identical structures']
df_ident

,mnxparent_label,parent_label,mnxchild_label,child_label,pair_key,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag


In [25]:
df_no_class = merged_df[merged_df['classification'] == 'Unclassified']
df_no_class

,mnxparent_label,parent_label,mnxchild_label,child_label,pair_key,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
16,MNXM101628,chebi:165778,MNXM34265,chebi:76150,"(chebi:76150, chebi:165778)",2791,2792,[chebi:165778],[chebi:76150],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0
20,MNXM102039,chebi:71255,MNXM149699,chebi:71251,"(chebi:71251, chebi:71255)",6316,6318,[chebi:71255],[chebi:71251],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0
21,MNXM102039,chebi:71255,MNXM149843,chebi:71252,"(chebi:71255, chebi:71252)",6316,6317,[chebi:71255],[chebi:71252],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0
57,MNXM10656,chebi:178383,MNXM1105855,chebi:58225,"(chebi:58225, chebi:178383)",3896,3904,[chebi:178383],[chebi:58225],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0
58,MNXM10656,chebi:178383,MNXM1364108,chebi:60332,"(chebi:60332, chebi:178383)",3896,3903,[chebi:178383],[chebi:60332],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5029,MNXM94083,chebi:182018,MNXM739919,chebi:84472,"(chebi:84472, chebi:182018)",4545,4546,[chebi:182018],[chebi:84472],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0
5037,MNXM9669,chebi:136812,MNXM52268,chebi:165792,"(chebi:136812, chebi:165792)",3052,3053,[chebi:136812],[chebi:165792],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Stereo and charge differ (complex); no classif...,v1.0
5046,MNXM9859,chebi:48946,MNXM31557,chebi:45525,"(chebi:45525, chebi:48946)",6513,6514,[chebi:48946],[chebi:45525],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0
5047,MNXM9879,chebi:173438,MNXM1409532,chebi:228256,"(chebi:228256, chebi:173438)",5084,5086,[chebi:173438],[chebi:228256],1,1,Unclassified,NaN,"{""confidence_bin"":null}",Parent-child stereochemical relationships must...,v1.0


The identical structure is a mistake of the pipeline. This needs to be investigated further to identify the root cause, its likelay an error of RDKit as the structures are clustered separately in an intial step. Should change the classification from identical structures to something more informative of what is actually going on.

Can use the extra_info column to understand the reason as to why there is No classification.

# Evaluation

Metrics used include precision, recall and F1 score. 

True positives: where mnx labels and stereomapper predictions agree

False positives: where they disagree

False negatives: cases where no prediction was made for that pair in question

In [27]:
# get the stats, false positives, false negatives, true positives
tp = merged_df[merged_df['classification'] == 'Parent-child']
fp = merged_df[merged_df['classification'] != 'Parent-child']
fn = df_parent_child[~df_parent_child['pair_key'].isin(merged_df['pair_key'])]

# convert to integers
tp = int(tp.shape[0])
fp = int(fp.shape[0])
fn = int(fn.shape[0])

print(f"True Positives: {tp}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

True Positives: 4338
False Positives: 715
False Negatives: 31


In [29]:
# get the rows related to false negatives
fn_rows = df_parent_child[~df_parent_child['pair_key'].isin(merged_df['pair_key'])]
fn_rows

,mnxparent_label,parent_label,mnxchild_label,child_label,pair_key
134,MNXM108773,chebi:67250,MNXM9356,chebi:67247,"(chebi:67250, chebi:67247)"
135,MNXM108773,chebi:67250,MNXM9367,chebi:65004,"(chebi:65004, chebi:67250)"
2446,MNXM1128640,chebi:167328,MNXM1413361,chebi:234482,"(chebi:167328, chebi:234482)"
3563,MNXM1369280,chebi:38144,MNXM1369281,chebi:38145,"(chebi:38144, chebi:38145)"
3564,MNXM1369280,chebi:38144,MNXM1369282,chebi:38146,"(chebi:38144, chebi:38146)"
3921,MNXM1371566,chebi:176889,MNXM734380,chebi:132823,"(chebi:176889, chebi:132823)"
3922,MNXM1371566,chebi:176889,MNXM1371567,chebi:27693,"(chebi:176889, chebi:27693)"
4197,MNXM1373631,chebi:181752,MNXM1370146,chebi:7469,"(chebi:7469, chebi:181752)"
4198,MNXM1373631,chebi:181752,MNXM1370147,chebi:91754,"(chebi:91754, chebi:181752)"
4436,MNXM1376850,chebi:189873,MNXM1376848,chebi:189871,"(chebi:189871, chebi:189873)"


In the cases of **false negatives**, two likely causes exist:

(1) The identifier in question has no structural representation on the chebi website / database

(2) stereomapper correctly / incorrectly clusters the structures in the pair together in the identity step, so no relationship can be assigned to them. 

These are the likely causes for the false negatives, as we removed wildcards in a previous step before entry to avoid their influence in this phase.

-----------------------------------------------------------------------------------------------------------

In terms of **false positives**, a few possible cases exist:

(1) The relationship between the structures in question do not fit into any classifications defined by the pipeline e.g., a case where structures A and B have different stereochemistry and protonation states.

(2) Misclassifications by MetaNetX / ChEBI , where stereomapper corrects them (in the cases mentioned previously)

(3) Cases where the pipeline assigns the incorrect relationship

For cases where we do not define the relationship between them, how should we handle this in the analysis? Perhaps present the two sets of results, one including the full set of false positives caused by "No classification" and those without ? If we go this route, we have to confirm they have no classification due to the definitions above. Should be easy - first gen inchikeys if 3rd layers differ we can confirm them to this definition, whatever is left over manually check for problems we did not think of.

In [30]:
actual_fp = fp - 2 # remove the 2 diastereomer cases where stereomapper is correct 

In [ ]:
# now lets calculate precision, recall and F1 score
## note this is the case where we count all "No classification" as false positives
precision = tp / (tp + actual_fp) if (tp + actual_fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1 Score: {f1_score:.2%}")

Precision: 85.88%
Recall: 99.29%
F1 Score: 92.10%


These are the initial set of results, not removing the false positives which we conclude to be due to limitations in the pipeline. Now lets confirm our suspicisions that these are due to not being able to fit into our defined classes due to charge differences. Then recompute stats above!

As such, even with inclusion of these false positives, the pipeline performs quite well in these cases.

In [39]:
# now let's remove those false positives we know are due to complex classification cases
# use extra_info column: drop rows whose extra_info mentions 'charge' or 'Radioactivity'
# cast to str to avoid issues with None / lists and use a regex to match either term
mask = merged_df['extra_info'].astype(str).str.contains(r'charge|Radioactivity', case=False, na=False)
df_filtered_no_class = merged_df[~mask]
df_filtered_no_class

,mnxparent_label,parent_label,mnxchild_label,child_label,pair_key,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
0,MNXM100051,chebi:187719,MNXM100344,chebi:188242,"(chebi:187719, chebi:188242)",6897,6898,[chebi:187719],[chebi:188242],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
1,MNXM10010,chebi:28254,MNXM1380605,chebi:198196,"(chebi:198196, chebi:28254)",6790,6791,[chebi:28254],[chebi:198196],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
2,MNXM10026,chebi:86041,MNXM1104681,chebi:62616,"(chebi:62616, chebi:86041)",2114,2116,[chebi:86041],[chebi:62616],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
3,MNXM10026,chebi:86041,MNXM9497,chebi:74272,"(chebi:74272, chebi:86041)",2114,2115,[chebi:86041],[chebi:74272],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
4,MNXM10039,chebi:50168,MNXM36497,chebi:50170,"(chebi:50170, chebi:50168)",253,257,[chebi:50168],[chebi:50170],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5045,MNXM9859,chebi:48946,MNXM1107135,chebi:145932,"(chebi:145932, chebi:48946)",6513,6515,[chebi:48946],[chebi:145932],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
5049,MNXM9987,chebi:27389,MNXM162802,chebi:57731,"(chebi:57731, chebi:27389)",4821,4823,[chebi:27389],[chebi:57731],1,1,Parent-child,84.0,"{""confidence_bin"":""medium""}",None,v1.0
5050,MNXM9987,chebi:27389,MNXM732376,chebi:58655,"(chebi:27389, chebi:58655)",4821,4822,[chebi:27389],[chebi:58655],1,1,Parent-child,84.0,"{""confidence_bin"":""medium""}",None,v1.0
5051,MNXM99960,chebi:186405,MNXM101734,chebi:195754,"(chebi:195754, chebi:186405)",1057,1058,[chebi:186405],[chebi:195754],1,1,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0


Vast majority of unclassified cases due to complex / ambiguous cases associated with charge diffs that aren't handled in StereoMapper.

Remaining are due to differences in isotopically labelled species, which are also not handled.

In [41]:
# check if any no classification remain
df_filtered_no_class['classification'].value_counts()

classification
Parent-child     4338
Diastereomers       3
Unresolved          1
Name: count, dtype: int64

32 remain, lets recalculate the precision, recall and f1 score

In [42]:
# get the stats, false positives, false negatives, true positives
tp_v2 = df_filtered_no_class[df_filtered_no_class['classification'] == 'Parent-child']
fp_v2 = df_filtered_no_class[df_filtered_no_class['classification'] != 'Parent-child']
fn_v2 = df_parent_child[~df_parent_child['pair_key'].isin(merged_df['pair_key'])] # still use merged df to find fns

# convert to integers
tp_v2 = int(tp_v2.shape[0])
fp_v2 = int(fp_v2.shape[0])
fn_v2 = int(fn_v2.shape[0])

# take away two diastereomer cases from false positives we know to be correct by molD
fp_v2 -= 2

print(f"True Positives: {tp_v2}")
print(f"False Positives: {fp_v2}")
print(f"False Negatives: {fn_v2}")

True Positives: 4338
False Positives: 2
False Negatives: 31


In [43]:
# now lets calculate precision, recall and F1 score
## note this is the case where we exclude "No classification" from false positives
precision_v2 = tp_v2 / (tp_v2 + fp_v2) if (tp_v2 + fp_v2) > 0 else 0
recall_v2 = tp_v2 / (tp_v2 + fn_v2) if (tp_v2 + fn_v2) > 0 else 0
f1_score_v2 = 2 * (precision_v2 * recall_v2) / (precision_v2 + recall_v2) if (precision_v2 + recall_v2) > 0 else 0

print(f"Precision: {precision_v2:.2%}")
print(f"Recall: {recall_v2:.2%}")
print(f"F1 Score: {f1_score_v2:.2%}")

Precision: 99.95%
Recall: 99.29%
F1 Score: 99.62%


Excellent improvement, 99% accuracy noted.

In [2]:
import os
from rdkit import Chem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Geometry import Point2D

# use chebi:38200 and chebi:27669 as false positive example
# first access local storage of molfiles 
chebi_strucs = "/media/JACK/stereomapper_structures/chebi_structures"
file1 = os.path.join(chebi_strucs, f"CHEBI_182512.sdf")
file2 = os.path.join(chebi_strucs, f"CHEBI_31053.sdf")

# Nicer coordinate generator
rdDepictor.SetPreferCoordGen(True)

# generate mols 
mol1 = Chem.MolFromMolFile(file1, sanitize=True)
mol2 = Chem.MolFromMolFile(file2, sanitize=True)

rdDepictor.Compute2DCoords(mol1)
rdDepictor.Compute2DCoords(mol2)

# create the drawer 
drawer = rdMolDraw2D.MolDraw2DSVG(800, 800)  # width, height
opts = drawer.drawOptions()
opts.addStereoAnnotation = True  # add stereochemistry annotations
opts.bondLineWidth = 2
opts.minFontSize = 14
opts.useBWAtomPalette()  # use black and white color scheme

# draw the mols
rdMolDraw2D.PrepareAndDrawMolecule(drawer, mol1, legend="CHEBI:182512")
drawer.FinishDrawing()
svg1 = drawer.GetDrawingText()
with open("/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/papers/stereomapper/v1/graphics/CHEBI_182512.svg", "w") as f:
    f.write(svg1)

# now for the second mol
drawer = rdMolDraw2D.MolDraw2DSVG(800, 800)  # width, height
opts = drawer.drawOptions()
opts.addStereoAnnotation = True  # add stereochemistry annotations
opts.bondLineWidth = 2
opts.minFontSize = 14
opts.useBWAtomPalette()  # use black and white color scheme 
rdMolDraw2D.PrepareAndDrawMolecule(drawer, mol2, legend="CHEBI:31053")
drawer.FinishDrawing()
svg2 = drawer.GetDrawingText()
with open("/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/papers/stereomapper/v1/graphics/CHEBI_31053.svg", "w") as f:
    f.write(svg2)

In [5]:
# first access local storage of molfiles 
vmh_strucs = "/media/JACK/stereomapper_structures/vmh_structures"
sabiork_strucs = "/media/JACK/stereomapper_structures/sabiork_structures"
file1 = os.path.join(vmh_strucs, f"r15bp.mol")
file2 = os.path.join(sabiork_strucs, f"sabiork_1470.mol")

# Nicer coordinate generator
rdDepictor.SetPreferCoordGen(True)

# generate mols 
mol1 = Chem.MolFromMolFile(file1, sanitize=True)
mol2 = Chem.MolFromMolFile(file2, sanitize=True)

rdDepictor.Compute2DCoords(mol1)
rdDepictor.Compute2DCoords(mol2)

# create the drawer 
drawer = rdMolDraw2D.MolDraw2DSVG(800, 800)  # width, height
opts = drawer.drawOptions()
opts.addStereoAnnotation = True  # add stereochemistry annotations
opts.bondLineWidth = 2
opts.minFontSize = 14
opts.useBWAtomPalette()  # use black and white color scheme

# draw the mols
rdMolDraw2D.PrepareAndDrawMolecule(drawer, mol1, legend="vmh:r15bp")
drawer.FinishDrawing()
svg1 = drawer.GetDrawingText()
with open("/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/papers/stereomapper/v1/graphics/vmh_r15bp.svg", "w") as f:
    f.write(svg1)

# now for the second mol
drawer = rdMolDraw2D.MolDraw2DSVG(800, 800)  # width, height
opts = drawer.drawOptions()
opts.addStereoAnnotation = True  # add stereochemistry annotations
opts.bondLineWidth = 2
opts.minFontSize = 14
opts.useBWAtomPalette()  # use black and white color scheme 
rdMolDraw2D.PrepareAndDrawMolecule(drawer, mol2, legend="sabiork:1470")
drawer.FinishDrawing()
svg2 = drawer.GetDrawingText()
with open("/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/papers/stereomapper/v1/graphics/sabirok_1470.svg", "w") as f:
    f.write(svg2)